# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [153]:
import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union, Optional

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [154]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [155]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [156]:
# Для начала реализуем сами критерии


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """
    _, counts = np.unique(x, return_counts=True)
    probabilities = counts / len(x)
    return 1 - np.sum(probabilities ** 2)
    


def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """
    _, counts = np.unique(x, return_counts=True)
    probabilities = counts / len(x)
    return -np.sum(probabilities * np.log2(probabilities))


def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    parent = np.concatenate((left_y, right_y))
    parent_impurity = criterion(parent)
    
    left_impurity = criterion(left_y) if len(left_y) > 0 else 0.0
    right_impurity = criterion(right_y) if len(right_y) > 0 else 0.0
    weighted_impurity = (len(left_y) * left_impurity + len(right_y) * right_impurity) / (len(right_y) + len(left_y))
    
    return parent_impurity - weighted_impurity

In [157]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [176]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.X = X
        self.y = y
        self._out_of_bag_X = np.ndarray([])
        self._out_of_bag_y = np.ndarray([])
        self.criterion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth if max_depth else float("inf")
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.sqrt(X.shape[1])) if max_features == "auto" else len(X[0])
        self.root = self._build_node(*self._choose_train_X())

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def feature_importance_j(self, j: int) -> float:
        return max(0, self._err_oob_j(j) - self._err_oob())
    
    def _err_oob(self) -> float:
        oob_result = self.predict(self._out_of_bag_X)
        return np.mean(self._out_of_bag_y != oob_result)

    def _err_oob_j(self, j: int) -> float:
        shuffle_j = np.array(self._out_of_bag_X.copy())
        np.random.shuffle(shuffle_j[:, j])
        shuffle_j_result = self.predict(shuffle_j)
        return np.mean(self._out_of_bag_y != shuffle_j_result)
        
    def _choose_train_X(self) -> Tuple[np.ndarray, np.ndarray]:
        train = random.choices(range(len(self.X)), k=len(self.X))
        oob = list(set(range(len(self.X))) - set(train))
        out_of_bag_X, out_of_bag_y = [], []
        for point in oob:
            out_of_bag_X.append(self.X[point])
            out_of_bag_y.append(self.y[point])
        self._out_of_bag_X, self._out_of_bag_y = out_of_bag_X, out_of_bag_y
        return np.array([self.X[point] for point in train]), np.array([self.y[point] for point in train])
    
    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int=0) -> DecisionTreeNode:
        if len(points) < 2 * self.min_samples_leaf or depth == self.max_depth:
            return DecisionTreeLeaf(classes)
        
        feature = self._get_best_feature(points, classes)
        left_x, right_x = self._question_point(feature, points, classes)
        left_y, right_y = self._question_class(feature, points, classes)
        if len(left_y) == 0:
            return DecisionTreeLeaf(right_y)
        if len(right_y) == 0:
            return DecisionTreeLeaf(left_y)
        return DecisionTreeInternalNode(feature, self._build_node(left_x, left_y, depth + 1), self._build_node(right_x, right_y, depth + 1))
        
    def _get_best_feature(self, points: np.ndarray, classes: np.ndarray) -> int:
        features = random.sample(range(len(points[0])), self.max_features)
        result_feature, best_gain_result = 0, float("-inf")
        for feature in features:
            left, right = self._question_class(feature, points, classes)
            gain_result = gain(left, right, self.criterion)
            if gain_result > best_gain_result:
                result_feature, best_gain_result = feature, gain_result
        return result_feature

    @staticmethod
    def _question_class(feature: int, points: np.ndarray, classes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        left, right = [], []
        for i, point in enumerate(points):
            if point[feature]:
                left.append(classes[i])
            else:
                right.append(classes[i])
        return np.array(left), np.array(right)

    
    @staticmethod
    def _question_point(feature: int, points: np.ndarray, classes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        left, right = [], []
        for i, point in enumerate(points):
            if point[feature]:
                left.append(point)
            else:
                right.append(point)
        return np.array(left), np.array(right)
        
    def predict_single_point(self, point: np.ndarray, node: Optional[DecisionTreeNode]=None) -> int:
        if node is None:
            node = self.root
        if isinstance(node, DecisionTreeLeaf):
            return node.max_class
        if point[node.split_dim]:
            return self.predict_single_point(point, node.left)
        return self.predict_single_point(point, node.right)
        
    def predict(self, points: np.ndarray) -> np.ndarray:
        return np.array([self.predict_single_point(point, self.root) for point in points])


### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [179]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []
        self._feature_number = 0

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features
    
    def feature_importance_j(self, j: int) -> float:
        result = 0
        for tree in self.estimators:
            delta = tree.feature_importance_j(j)
            result += delta
        return result / self._n_estimators

    
    def feature_importance(self) -> np.ndarray:
        result = []
        for j in range(self._feature_number):
            result.append(self.feature_importance_j(j))

        if np.sum(result) > 0:
            result /= np.sum(result)
        
        return result
    
    
    def fit(self, X, y):
        self._feature_number = len(X[0])
        for _ in range(self._n_estimators):
            self.estimators.append(DecisionTree(X, y, self._criterion, self._max_depth, self._min_samples_leaf, self._max_features))

    def predict(self, X):
        predicts = np.array([tree.predict(X) for tree in self._estimators])
        result = []
        for tree_preds in predicts.T:
            result.append(mode(tree_preds))
        return result
        

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [160]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    return rfc.feature_importance()


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [180]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.00313275 0.00617325 0.23022527 0.24092433 0.51117186 0.00837254]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [181]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [182]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [183]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7238335435056746
Most important features:
1. ovsyanochan
2. 4ch
3. rhymes
4. mudakoff
5. styd.pozor
6. rapnewrap
7. reflexia_our_feelings
8. pravdashowtop
9. bot_maxim
10. dayvinchik
11. ne1party
12. iwantyou
13. tumblr_vacuum
14. pixel_stickers
15. fuck_humor
16. pozor
17. soverwenstvo.decora
18. leprum
19. ne.poverish
20. rem_shkola


#### Пол

In [184]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8373266078184111
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. 4ch
5. zerofat
6. mudakoff
7. 9o_6o_9o
8. be.beauty
9. i_d_t
10. thesmolny
11. reflexia_our_feelings
12. femalemem
13. cook_good
14. sh.cook
15. be.women
16. rapnewrap
17. bot_maxim
18. academyofman
19. beauty
20. soverwenstvo.decora


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [185]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

0:	learn: 0.4239784	total: 50.8ms	remaining: 457ms
1:	learn: 0.1037410	total: 51.4ms	remaining: 206ms
2:	learn: 0.0475388	total: 51.9ms	remaining: 121ms
3:	learn: 0.0271089	total: 52.3ms	remaining: 78.5ms
4:	learn: 0.0157713	total: 52.7ms	remaining: 52.7ms
5:	learn: 0.0102860	total: 53.1ms	remaining: 35.4ms
6:	learn: 0.0074490	total: 53.6ms	remaining: 23ms
7:	learn: 0.0051865	total: 54ms	remaining: 13.5ms
8:	learn: 0.0041190	total: 54.5ms	remaining: 6.05ms
9:	learn: 0.0032379	total: 54.9ms	remaining: 0us
Accuracy: 1.0
Importance: [ 0.          0.         22.24583462 28.33697139 49.41719399  0.        ]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [186]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [198]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score, best_params = None, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):
        params = { "learning_rate":lr,
                   "max_depth":md,
                   "min_data_in_leaf":msl,
                   "loss_function":'MultiClass',
                   "logging_level":'Silent'}
        cb_model = CatBoostClassifier(**params)
        cb_model.fit(X_train, y_train)
        y_pred = cb_model.predict(X_eval)
        score = accuracy_score(y_pred, y_eval)
        if not best_score or score > best_score:
            best_score, best_params = score, params

    return best_params, best_score

#### Возраст

In [199]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075),
  'max_depth': 7,
  'min_data_in_leaf': 1,
  'loss_function': 'MultiClass',
  'logging_level': 'Silent'},
 np.float64(0.7610371408549405))

In [200]:
cb_model = CatBoostClassifier(**best_params)
cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7288776796973518
Most important features:
1. mudakoff
2. 4ch
3. dayvinchik
4. exclusive_muzic
5. rhymes
6. rapnewrap
7. kino_mania
8. styd.pozor
9. ovsyanochan
10. fuck_humor


#### Пол

In [201]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075),
  'max_depth': 1,
  'min_data_in_leaf': 1,
  'loss_function': 'MultiClass',
  'logging_level': 'Silent'},
 np.float64(0.8710581639803784))

In [203]:
cb_model = CatBoostClassifier(**best_params)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8738965952080706
Most important features:
1. 40kg
2. modnailru
3. girlmeme
4. mudakoff
5. i_d_t
6. 9o_6o_9o
7. be.beauty
8. zerofat
9. thesmolny
10. igm
